## Load Library

In [44]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn
from datetime import datetime

#set number to .2 decimal
pd.options.display.float_format = '{:,.2f}'.format

## Data Preprocessing

Dataset for this analysis is from https://www.kaggle.com/datasets/carrie1/ecommerce-data?resource=download

In [6]:
#load dataset
df = pd.read_csv('assets/data.csv', encoding='ISO-8859-1') #this dataset has some problem with default UTF-8 encoding

#see data information
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


### Handle Missing Value

In [5]:
print(df.isnull().sum())
df.head()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,"17,850.00",United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,"17,850.00",United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,"17,850.00",United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,"17,850.00",United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,"17,850.00",United Kingdom


There are 135k row transaction with Null Customer ID. There are 2 ways to deal with this; impute or drop the null values. In this analysis, because we want to identify customer segment and there are no other consideration, we go with drop the null values

In [88]:
df_1 = df.dropna(subset='CustomerID')
df_1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 406829 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    406829 non-null  object 
 1   StockCode    406829 non-null  object 
 2   Description  406829 non-null  object 
 3   Quantity     406829 non-null  int64  
 4   InvoiceDate  406829 non-null  object 
 5   UnitPrice    406829 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      406829 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 27.9+ MB


There are no missing value after we drop row null value by CustomerID

### Changing Data Types

In [89]:
df_1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 406829 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    406829 non-null  object 
 1   StockCode    406829 non-null  object 
 2   Description  406829 non-null  object 
 3   Quantity     406829 non-null  int64  
 4   InvoiceDate  406829 non-null  object 
 5   UnitPrice    406829 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      406829 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 27.9+ MB


Changing data types:
* Invoice Date  : datetime
* CustomerID    : int

In [90]:
#changing invoice date into datetime type
df_1['InvoiceDate'] = pd.to_datetime(df_1['InvoiceDate'])

#changing CustomerID to object
df_1['CustomerID'] = df_1['CustomerID'].astype('int')

C:\Users\satri\AppData\Local\Temp\ipykernel_3112\1075245377.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['InvoiceDate'] = pd.to_datetime(df_1['InvoiceDate'])
C:\Users\satri\AppData\Local\Temp\ipykernel_3112\1075245377.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['CustomerID'] = df_1['CustomerID'].astype('int')


In [91]:
df_1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 406829 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    406829 non-null  object        
 1   StockCode    406829 non-null  object        
 2   Description  406829 non-null  object        
 3   Quantity     406829 non-null  int64         
 4   InvoiceDate  406829 non-null  datetime64[ns]
 5   UnitPrice    406829 non-null  float64       
 6   CustomerID   406829 non-null  int64         
 7   Country      406829 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(2), object(4)
memory usage: 27.9+ MB


### Handle Return Transaction

In [92]:
df_1.describe()

,Quantity,InvoiceDate,UnitPrice,CustomerID
count,"406,829.00",406829,"406,829.00","406,829.00"
mean,12.06,2011-07-10 16:30:57.879207424,3.46,"15,287.69"
min,"-80,995.00",2010-12-01 08:26:00,0.00,"12,346.00"
25%,2.00,2011-04-06 15:02:00,1.25,"13,953.00"
50%,5.00,2011-07-31 11:48:00,1.95,"15,152.00"
75%,12.00,2011-10-20 13:06:00,3.75,"16,791.00"
max,"80,995.00",2011-12-09 12:50:00,"38,970.00","18,287.00"
std,248.69,NaN,69.32,"1,713.60"


In [93]:
len_return = len(df_1[df_1['Quantity'] < 0])
len_data = len(df_1)
ratio_return = len_return/len_data * 100.0

print(f"Jumlah Return sebanyak {len_return} atau {round(ratio_return,2)}% dari total data")

Jumlah Return sebanyak 8905 atau 2.19% dari total data


In [94]:
#check return data
df_return = df_1[df_1['Quantity'] < 0]

print(df_return.InvoiceNo.str[:1].unique())

['C']


In [95]:
df_return['InvoiceNo_initial'] = df_return['InvoiceNo'].str[1:]

df_initial = df_1[df_1['InvoiceNo'].isin(list(df_return['InvoiceNo_initial'].unique()))]

print(len(df_return))
print(len(df_initial))

8905
0


C:\Users\satri\AppData\Local\Temp\ipykernel_3112\1710432458.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_return['InvoiceNo_initial'] = df_return['InvoiceNo'].str[1:]


Tidak ada initial transaction ditemukan pada data return, dikarenakan data return tidak terlalu berpengaruh pada data analysis kali ini, maka kita drop saja data returnnya

In [96]:
#drop data return
df_2 = df_1[
    df_1['Quantity'] >= 0
]

In [97]:
# Separate numeric and categorical columns
numeric_cols = df_2.select_dtypes(include=['number'])  # Select numeric columns
categorical_cols = df_2.select_dtypes(exclude=['number'])  # Select categorical columns

# Describe numeric columns
print("Numeric Columns Description:")
print(numeric_cols.describe())

# Describe categorical columns
print("\nCategorical Columns Description:")
print(categorical_cols.describe())

Numeric Columns Description:
        Quantity  UnitPrice  CustomerID
count 397,924.00 397,924.00  397,924.00
mean       13.02       3.12   15,294.32
std       180.42      22.10    1,713.17
min         1.00       0.00   12,346.00
25%         2.00       1.25   13,969.00
50%         6.00       1.95   15,159.00
75%        12.00       3.75   16,795.00
max    80,995.00   8,142.75   18,287.00

Categorical Columns Description:
                         InvoiceDate
count                         397924
mean   2011-07-10 23:43:36.912475648
min              2010-12-01 08:26:00
25%              2011-04-07 11:12:00
50%              2011-07-31 14:39:00
75%              2011-10-20 14:33:00
max              2011-12-09 12:50:00


### Unit Price == 0

In [113]:
print(df_2[df_2['UnitPrice'] == 0].head())
print(f"Ada {len(df_2[df_2['UnitPrice'] == 0])} data yang unit pricenya 0")

      InvoiceNo StockCode                   Description  Quantity  \
9302     537197     22841  ROUND CAKE TIN VINTAGE GREEN         1   
33576    539263     22580  ADVENT CALENDAR GINGHAM SACK         4   
40089    539722     22423      REGENCY CAKESTAND 3 TIER        10   
47068    540372     22090       PAPER BUNTING RETROSPOT        24   
47070    540372     22553        PLASTERS IN TIN SKULLS        24   

              InvoiceDate  UnitPrice  CustomerID         Country  
9302  2010-12-05 14:02:00       0.00       12647         Germany  
33576 2010-12-16 14:36:00       0.00       16560  United Kingdom  
40089 2010-12-21 13:45:00       0.00       14911            EIRE  
47068 2011-01-06 16:41:00       0.00       13081  United Kingdom  
47070 2011-01-06 16:41:00       0.00       13081  United Kingdom  
Ada 40 data yang unit pricenya 0


In [ ]:
#product yang nilai Unit Pricenya 0
df_2[df_2['UnitPrice'] == 0].groupby('Description').agg({'InvoiceNo':'count','Quantity':'sum'}).reset_index()

,Description,InvoiceNo,Quantity
0,OVAL WALL MIRROR DIAMANTE,1,1
1,36 FOIL STAR CAKE CASES,1,144
2,ADVENT CALENDAR GINGHAM SACK,1,4
3,ASSTD DESIGN 3D PAPER STICKERS,1,12540
4,BISCUIT TIN VINTAGE CHRISTMAS,1,216
5,CERAMIC BOWL WITH LOVE HEART DESIGN,1,36
6,CHILDREN'S APRON DOLLY GIRL,1,1
7,CHILDS BREAKFAST SET CIRCUS PARADE,1,1
8,CHRISTMAS PUDDING TRINKET POT,1,12
9,FAIRY CAKES NOTEBOOK A6 SIZE,1,16


In [131]:
#remove unit price 0
df_3 = df_2[df_2['UnitPrice'] > 0]

## Exploratory Data Analysis

In [138]:
print(f"Transaksi dari tanggal {df_3['InvoiceDate'].dt.date.min()} sampai dengan tanggal {df_3['InvoiceDate'].dt.date.max()}")
print(f"="*100)
print(f"Jumlah Transaksi sebanyak {len(df_3)}")
print(f"="*100)
print(f"Ada {df_3['CustomerID'].nunique()} yang berbelanja selama periode tersebut")

Transaksi dari tanggal 2010-12-01 sampai dengan tanggal 2011-12-09
Jumlah Transaksi sebanyak 397884
Ada 4338 yang berbelanja selama periode tersebut
